In [11]:
import re
import json

def parse_markdown_to_json(markdown_file_path, output_json_path):
    """
    Parses a markdown file containing questions and answers,
    extracts them, and saves them to a JSON file.

    Each question is expected to start with a marker like 【题 XX】, [題 XX],
    【图XX】, 【原 XX】, or [理 XX].
    The question text is the content between this marker and the
    start of the answer section (marked by 【分析】 or 【解】).
    The answer text includes the answer marker and the subsequent content
    until the next question marker.
    """
    try:
        with open(markdown_file_path, 'r', encoding='utf-8') as f:
            markdown_content = f.read()
    except FileNotFoundError:
        print(f"Error: The file '{markdown_file_path}' was not found.")
        print("Please ensure the markdown file is in the same directory as the script,")
        print("or provquestion_numbere the correct path.")
        return
    except Exception as e:
        print(f"Error reading file '{markdown_file_path}': {e}")
        return

    questions_data = []
    
    # Regex to find all question starting markers.
    # This pattern captures:
    # 【题...】 (e.g., 【题 21】)
    # [題...] (e.g., [題 2])
    # 【图...】 (e.g., 【图24】) - Assuming these are questions based on your document structure
    # 【原...】 (e.g., 【原 41】)
    # [理...] (e.g., [理 23】)
    problem_marker_pattern = r'(【题\s*[^】]+】|\[題\s*[^\]]+\]|【图\s*[^】]+】|【原\s*[^】]+】|\[理\s*[^\]]+\])'
    
    # Find all starting positions of question markers
    matches = list(re.finditer(problem_marker_pattern, markdown_content))
    
    if not matches:
        print("No questions found. Please check the markers in your markdown file.")
        return

    for i, match in enumerate(matches):
        q_question_number_full_marker = match.group(1)  # The full marker, e.g., "【题 21】"
        
        # The actual content of the current question block starts after its marker
        content_start_index = match.end()
        
        # The end of the current question block is the start of the next question's marker,
        # or the end of the file if this is the last question.
        if i + 1 < len(matches):
            block_end_index = matches[i+1].start()
        else:
            block_end_index = len(markdown_content)
            
        current_block_content = markdown_content[content_start_index:block_end_index].strip()
        
        # Clean up the question_number: remove brackets and leading/trailing whitespace
        # e.g., "【题 21】" becomes "题 21"
        q_question_number = re.sub(r'[【】\[\]]', '', q_question_number_full_marker).strip()

        question_text = ""
        answer_text = ""
        
        # Try to find the start of the answer section (【分析】 or 【解】)
        analysis_marker_str = "【分析】"
        solution_marker_str = "【解】"
        
        analysis_pos = current_block_content.find(analysis_marker_str)
        solution_pos = current_block_content.find(solution_marker_str)
        
        answer_marker_pos = -1
        
        # Determine the earliest position of an answer marker
        if analysis_pos != -1 and solution_pos != -1:
            answer_marker_pos = min(analysis_pos, solution_pos)
        elif analysis_pos != -1:
            answer_marker_pos = analysis_pos
        elif solution_pos != -1:
            answer_marker_pos = solution_pos
            
        if answer_marker_pos != -1:
            # Question is text before the answer marker
            # question_text = current_block_content[:answer_marker_pos].strip()
            question_text = current_block_content[:answer_marker_pos]

            # Answer includes the marker itself and the rest of the block
            # answer_text = current_block_content[answer_marker_pos:].strip()
            answer_text = current_block_content[answer_marker_pos:]

        else:
            # If no specific answer marker is found, assume the entire block
            # after the question_number marker is the question text. This might happen if
            # a question (e.g., a 【图X】 item) doesn't have a separate analysis/solution.
            question_text = current_block_content
            answer_text = "" # No separate answer section question_numberentified

        questions_data.append({
            "question_number": q_question_number,
            "question": str(question_text),
            "answer": str(answer_text)
        })

    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(questions_data, f, ensure_ascii=False, indent=4)
        print(f"Successfully extracted {len(questions_data)} questions to '{output_json_path}'")
    except Exception as e:
        print(f"Error writing JSON to file '{output_json_path}': {e}")

question_text = ''
# 1. Save the markdown content you provquestion_numbered into a file named "questions.md"
#    in the same directory as this Python script.
# 2. Run this script. It will generate "questions_extracted.json".

markdown_file = "questions.md"  # Input file name
json_file = "questions_extracted.json" # Output file name

parse_markdown_to_json(markdown_file, json_file)

Successfully extracted 98 questions to 'questions_extracted.json'


In [19]:
import sys
import pdfplumber
import os
import json
import time
def extract_pdf_text(pdf_path, save_dir=None):
    """
    提取PDF文本并保存到文件中
    
    Args:
        pdf_path (str): PDF文件路径
        save_dir (str, optional): 保存目录，默认为PDF同目录
        
    Returns:
        str: 保存的文本文件路径
    """
    # 如果未提供保存目录，使用PDF所在目录
    if save_dir is None:
        save_dir = os.path.dirname(pdf_path)
    
    # 创建保存目录（如果不存在）
    os.makedirs(save_dir, exist_ok=True)
    
    # 生成保存文件名（使用PDF文件名+.json）
    pdf_filename = os.path.basename(pdf_path)
    pdf_name_without_ext = os.path.splitext(pdf_filename)[0]
    save_path = os.path.join(save_dir, f"{pdf_name_without_ext}.json")
    
    # 检查是否已经提取过文本
    if os.path.exists(save_path):
        print(f"找到已提取的文本文件: {save_path}")
        return save_path
    
    # 开始提取
    print(f"开始提取PDF文本...")
    start_time = time.time()
    
    extracted_data = {
        "pdf_path": pdf_path,
        "extraction_time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_pages": 0,
        "pages": {}
    }
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            extracted_data["total_pages"] = total_pages
            
            print(f"PDF总页数: {total_pages}")
            
            for page_num, page in enumerate(pdf.pages):
                current_page = page_num + 1  # 页码从1开始
                print(f"正在提取第 {current_page}/{total_pages} 页...")
                
                # 提取文本
                text = page.extract_text() or ""
                extracted_data["pages"][str(current_page)] = text
        
        # 保存提取结果
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, ensure_ascii=False, indent=2)
        
        end_time = time.time()
        print(f"文本提取完成！用时 {end_time - start_time:.2f} 秒")
        print(f"文本已保存至: {save_path}")
        
        return save_path
        
    except Exception as e:
        print(f"提取PDF文本时发生错误: {e}")
        return None

def find_text_in_saved_pdf(text_file_path, search_text):
    """
    在已保存的PDF文本中查找指定文本
    
    Args:
        text_file_path (str): 保存的文本文件路径
        search_text (str): 需要查找的文本
    
    Returns:
        list: 包含文本出现页码的列表（页码从1开始）
    """
    result_pages = []
    
    try:
        # 读取保存的文本文件
        with open(text_file_path, 'r', encoding='utf-8') as f:
            pdf_data = json.load(f)
        
        # 获取PDF总页数
        total_pages = pdf_data.get("total_pages", 0)
        pages_data = pdf_data.get("pages", {})
                
        # 遍历每一页
        for page_num in range(1, total_pages + 1):
            # 获取当前页面文本
            text = pages_data.get(str(page_num), "")
            
            # 检查搜索文本是否在当前页面
            if search_text in text:
                result_pages.append(page_num)
    
    except Exception as e:
        print(f"查找文本时发生错误: {e}")
    
    return result_pages

def is_chinese_char(char):
    """
    判断一个字符是否为常用的汉字。
    这里使用的是基本的CJK统一表意文字区段 (U+4E00 至 U+9FFF)。
    如果需要更广泛的汉字支持（例如扩展区A、B等），可以扩展此处的Unicode范围。
    """
    return '\u4e00' <= char <= '\u9fff'

def get_consecutive_chinese_chars(text):
    """
    从输入文本中随机查找一个由5个连续汉字组成的子字符串。
    """
    if not text or len(text) < 5:
        return ""  # 如果文本为空或长度小于10，则返回空字符串

    possible_substrings = []
    # 遍历所有可能的10字符子串的起始位置
    for i in range(len(text) - 4):  # 确保子字符串长度为10
        substring = text[i:i+5]
        # 检查子字符串中的所有字符是否都是汉字
        if all(is_chinese_char(char) for char in substring):
            possible_substrings.append(substring)
        
    return possible_substrings






# 定义 PDF 文件路径
pdf_path = r"物理学难题集萃(增订本)【舒幼生等】_part1(OCR).pdf"
json_path = r"物理学难题集萃(增订本)【舒幼生等】_part1(OCR).json"

# extract_pdf_text(pdf_path, save_dir=json_path)

condition = "1 mol 单原子分子理想气体所经准静态循环过程是如热图2-21-1所示的圆，有关参量已在热图2-21-1 中标明。"

page

PDF总页数: 570
PDF总页数: 570
PDF总页数: 570
PDF总页数: 570
PDF总页数: 570


403

In [20]:
pages

[403, 372, 403, 398, 403]

In [8]:
possible_substrings

['单原子分子',
 '原子分子理',
 '子分子理想',
 '分子理想气',
 '子理想气体',
 '理想气体所',
 '想气体所经',
 '气体所经准',
 '体所经准静',
 '所经准静态',
 '经准静态循',
 '准静态循环',
 '静态循环过',
 '态循环过程',
 '循环过程是',
 '环过程是如',
 '过程是如热',
 '程是如热图',
 '有关参量已',
 '关参量已在',
 '参量已在热',
 '量已在热图']

In [ ]:
from openai import OpenAI
import json
import os
import time
import numpy as np
import datetime
import subprocess
from dotenv import load_dotenv
import random  # PyMuPDF
import re
import pandas as pd
import pdfplumber
from collections import Counter
import random

# 如果存在 .env 文件,从中加载环境变量
load_dotenv()
# 配置代理（如需要）
# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# client = OpenAI(
#     api_key=os.getenv("openai_api"),
# )
# MODEL = "gpt-4o-mini"
# INPUT_JSON_FILE = "questions_extracted.json"
# OUTPUT_JSON_FILE = "questions_transformed_gpt.json"
# CHECKPOINT_FILE = "last_call_gpt.npy"  # 添加检查点文件名

client = OpenAI(
    api_key=os.getenv("deepseek_api"),
    base_url="https://api.deepseek.com",
)
MODEL = "deepseek-chat"

INPUT_JSON_FILE = "questions_extracted.json"
OUTPUT_JSON_FILE = "questions_transformed_ds.json"
CHECKPOINT_FILE = "last_call_ds.npy"  # 添加检查点文件名

# --- 提示词 ---

prompt_extra = """
给定一个问题及其参考答案,按照以下规则提取出['question_number','condition','specific_questions','solution','final_answer'],并返回一个JSON列表。

其中:
"question_number": 字符串,题号
"condition": 原题的题干,直接复制原题的Latex内容,然后参考转换规则修改,
"specific_questions": 原题的设问,直接复制原题的Latex内容,然后参考转换规则修改,
"solution": 子问的逐步solution,改写为独立,不包含图示,且不直接引用其他部分结论。如需前部结果,应作为已知条件说明,
"final_answer": 一个数值或公式,不要任何汉字、条件、单位, 不要出现 '=','\n','\box'


转换规则:
1. 输出格式: 输出必须是一个JSON格式{"result":[字典列表]}, 每个字典必须是latex格式,确保能用latex编译器编译通过
2. 单个子问对应一个 JSON 对象: 若原题包含多个子问题(如 1., 2., a., b.),拆分为多个对象,question_number 用原 question_number.1/2/3/4...
3. 子问题独立: 每个对象必须尽量自包含。"condition"和"solution"避免出现"由第 1 部分得出"等表述,若需前部结果,请将结果写入该部分的"condition"
4. 转换选填题: 若"specific_questions"为选择题或填空题,需转换为计算题并给出数值或公式;不可行则跳过。
5. 内容完整: 保留题目核心物理概念和数值,改写不改变实质。
6. 结构: 严格包含"question_number"、"condition"、"specific_questions"、"solution"、"final_answer"五个字段。
7. 需要删除的内容 :【多余的补位符号(*#?-)】【题目序号】【如图xx所示】【某个大学】【某个省份】,"final_answer"里的汉字、单位、条件
8. 需要核对的内容 : 核对原题目和答案汉字的正确性:人射 vs 入射、代人 vs 代入、收玫 vs 收敛;latex代码的正确性:$\\overrightarrow{{AB}}$ vs $\\overline{{AB}}$
9. 需要修改的格式 : \\[替换为 换行$$,\\]替换为 $$换行,\\(替换为$,\\)替换为$;

EXAMPLE INPUT:
QURSTION:
一个静态球对称相对论星球的线元可写成

$$
\mathrm{d} s^{2}=e^{2 \Phi} \mathrm{~d} t^{2}-\frac{\mathrm{d} r^{2}}{1-r^{2} Y(r)}-r^{2} \mathrm{~d} \theta^{2}-r^{2} \sin ^{2} \theta \mathrm{~d} \varphi^{2}
$$

其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.
(1)用该线元中的二个已知函数 $\Phi$ 和 $Y$ 给出此星球的质量以及远距离的观察者所测得的在星球中心的红移, 用已知函数 $\Phi 、 Y$ 和 $n$ 表示该星球中的总强子数.
(2)用已知函数给出静力平衡条件.      

ANSWER:
解答 (1) 星球的质量

$$
M=\int_{0}^{R} \sqrt{-g_{r r} \rho}(r) \mathrm{d} r \int_{0}^{\pi} \sqrt{-g_{\theta \theta}} \mathrm{d} \theta \int_{0}^{2 \pi} \sqrt{-g_{\varphi \varphi}} \mathrm{d} \varphi
$$

所以

$$
M=4 \pi \int_{0}^{R} \rho(r) \frac{1}{\sqrt{1-r^{2} Y(r)}} r^{2} \mathrm{~d} r
$$

类似质量,星球中的总强子数,

$$
N=4 \pi \int_{0}^{R} n(r) \frac{r^{2} \mathrm{~d} r}{\sqrt{1-r^{2} Y(r)}}
$$

星球中心到星体表面的红移

$$
\frac{\lambda_{2}}{\lambda_{1}}=\left[\frac{g_{00}\left(x_{1}\right)}{g_{00}\left(x_{2}\right)}\right]^{1 / 2}=\mathrm{e}^{\Phi(r)-\Phi(0)}
$$

星体表面到远距离观察者引力导致红移

$$
\frac{\lambda_{3}}{\lambda_{2}}=\sqrt{1-\frac{2 G M}{c^{2} R}}
$$

远距离的观察者所测得的在星球中心的红移

$$
Z=\frac{\Delta \lambda}{\lambda}=\frac{\lambda_{3}}{\lambda_{1}}-1=\frac{\lambda_{2}}{\lambda_{1}} \frac{\lambda_{3}}{\lambda_{2}}-1=\sqrt{1-\frac{2 G M}{c^{2} R}} \mathrm{e}^{\Phi(r)-\Phi(0)}-1
$$

(2) 静力平衡的 Euler 方程为

$$
-\frac{\partial p}{\partial x^{\mu}}=(p+\rho) \frac{\partial}{\partial x^{\mu}} \ln \left(g_{00}\right)^{1 / 2}
$$

所以由本题的条件直接可得

$$
-\frac{\mathrm{d} p(r)}{\mathrm{d} r}=[p(r)+\rho(r)] \frac{\mathrm{d}}{\mathrm{~d} r} \ln \left[\mathrm{e}^{2 \Phi}\right]^{1 / 2}=[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}
$$

EXAMPLE JSON OUTPUT:
{"result":[{
"question_number": "7.7.1" ,
"condition":"一个静态球对称相对论星球的线元可写成

$$
\mathrm{d} s^{2}=e^{2 \Phi} \mathrm{~d} t^{2}-\frac{\mathrm{d} r^{2}}{1-r^{2} Y(r)}-r^{2} \mathrm{~d} \theta^{2}-r^{2} \sin ^{2} \theta \mathrm{~d} \varphi^{2}
$$

其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.",
"specific_questions":"用该线元中的二个已知函数 $\Phi$ 和 $Y$ 给出此星球的质量以及远距离的观察者所测得的在星球中心的红移, 用已知函数 $\Phi 、 Y$ 和 $n$ 表示该星球中的总强子数.",
"solotion":"星球的质量

$$
M=\int_{0}^{R} \sqrt{-g_{r r} \rho}(r) \mathrm{d} r \int_{0}^{\pi} \sqrt{-g_{\theta \theta}} \mathrm{d} \theta \int_{0}^{2 \pi} \sqrt{-g_{\varphi \varphi}} \mathrm{d} \varphi
$$

所以

$$
M=4 \pi \int_{0}^{R} \rho(r) \frac{1}{\sqrt{1-r^{2} Y(r)}} r^{2} \mathrm{~d} r
$$

类似质量，星球中的总强子数，

$$
N=4 \pi \int_{0}^{R} n(r) \frac{r^{2} \mathrm{~d} r}{\sqrt{1-r^{2} Y(r)}}
$$

星球中心到星体表面的红移

$$
\frac{\lambda_{2}}{\lambda_{1}}=\left[\frac{g_{00}\left(x_{1}\right)}{g_{00}\left(x_{2}\right)}\right]^{1 / 2}=\mathrm{e}^{\Phi(r)-\Phi(0)}
$$

星体表面到远距离观察者引力导致红移

$$
\frac{\lambda_{3}}{\lambda_{2}}=\sqrt{1-\frac{2 G M}{c^{2} R}}
$$

远距离的观察者所测得的在星球中心的红移

$$
Z=\frac{\Delta \lambda}{\lambda}=\frac{\lambda_{3}}{\lambda_{1}}-1=\frac{\lambda_{2}}{\lambda_{1}} \frac{\lambda_{3}}{\lambda_{2}}-1=\sqrt{1-\frac{2 G M}{c^{2} R}} \mathrm{e}^{\Phi(r)-\Phi(0)}-1
$$
",
"final_answer":"$$
Z=\frac{\Delta \lambda}{\lambda}=\frac{\lambda_{3}}{\lambda_{1}}-1=\frac{\lambda_{2}}{\lambda_{1}} \frac{\lambda_{3}}{\lambda_{2}}-1=\sqrt{1-\frac{2 G M}{c^{2} R}} \mathrm{e}^{\Phi(r)-\Phi(0)}-1
$$"},
{
"question_number": "7.7.1" ,
"condition":"一个静态球对称相对论星球的线元可写成

$$
\mathrm{d} s^{2}=e^{2 \Phi} \mathrm{~d} t^{2}-\frac{\mathrm{d} r^{2}}{1-r^{2} Y(r)}-r^{2} \mathrm{~d} \theta^{2}-r^{2} \sin ^{2} \theta \mathrm{~d} \varphi^{2}
$$

其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.",
"specific_questions": "用已知函数给出静力平衡条件.",
"solution": "静力平衡的 Euler 方程为

$$
-\frac{\partial p}{\partial x^{\mu}}=(p+\rho) \frac{\partial}{\partial x^{\mu}} \ln \left(g_{00}\right)^{1 / 2}
$$

所以由本题的条件直接可得

$$
-\frac{\mathrm{d} p(r)}{\mathrm{d} r}=[p(r)+\rho(r)] \frac{\mathrm{d}}{\mathrm{~d} r} \ln \left[\mathrm{e}^{2 \Phi}\right]^{1 / 2}=[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}
$$",
"final_answer":"$$[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}$$"}
]}

现在你已经知道了转换规则,请按照转换规则将给定的问题转换为JSON列表。
"""

prompt_question = """
给定一个物理问题, 请给出该问题的 ['适合年级', '子学科']

注意:
1. 适合年级只能是大学的题目, 分析题目适合大学几年级学生做, 直接写[大一/大二/大三/大四]
2. 子学科是物理具体的学科分支,如力学、电磁学等

EXAMPLE JSON OUTPUT:
{
"适合年级":"大一",
"子学科":"电磁学"
}

现在给定问题,请严格JSON式回答。你的回答只需要包括适合年级和子学科,不要添加其他任何内容。
问题:
"""

prompt_answer = """
给定一个物理问题及参考答案, 请给出该问题的 ['考察知识点', '分析过程']

注意:
1. 考察知识点需要按照['知识点1','知识点2',...]格式列出题目涉及的主要知识点
2. 分析过程需要凝练参考答案的分析过程,用一段话简洁地总结参考答案的思考过程
3. 分析过程应该尽可能凝练简洁,尽可能不包含公式, 如果一定要包含某条公式, 需要给出公式的LaTex格式

EXAMPLE JSON OUTPUT:
{
"考察知识点":['库伦定律', '受力分析', '共力点'],
"分析过程": "通过分析可移动小环在倾斜杆上最高点与最低点的受力情况,利用沿杆方向的力平衡条件（结合库仑定律和静摩擦力公式）建立方程,分别求解出能停住的上下临界位置的$y$坐标,从而确定其范围。"
}

现在给定问题,请严格JSON按照格式回答。你的回答只需要包括考察知识点和分析过程,不要添加其他任何内容。
问题:
"""

prompt_wrong = """
给定一个物理问题、正确答案以及错误答案, 请给出该问题的 ['错误解题方法','易错点']

注意:
1. 【错误解题方法】即错误答案,需要对比正确答案与错误答案的步骤,在错误答案中标注错误位置,标注格式和方法如下:【@Wrong_Step:reason@易错点】
2. 【易错点】需要用一段话总结易错的地方,需要包含【@Wrong_Step:reason@易错点】中的全部易错点,以及其他的易错点(即使错误答案没有犯这个错)

EXAMPLE JSON OUTPUT:
{
"错误解题方法":"...该级数为复数对数级数:$\sum_{n-1}^{\infty}\dfrac{o^{inx}}{n} = -\ln(1-e^{ix})$(当$e^{ix}=1$且$x \neq 2k\pi$时收敛)。
代入得:@rong_Step:reason@误用发散级数的导数积分
$S(x)=\text{Im} \left[-\ln(1-o{ix})\right]$...
",
"易错点":"...,误用发散级数的导数积分,..."
}

现在给定问题,请严格JSON按照格式回答。你的回答只需要包括错误解题方法和易错点,不要添加其他任何内容。
问题:
"""

prompt_judge = """
给定一个物理问题、正确答案以及三个学生答案, 请判断学生答案是否正确。

INPUT:
物理问题:...
正确答案:...
学生1答案:...
学生2答案:...
学生3答案:...


EXAMPLE OUTPUT:
{
"学生1":"正确/错误",
"学生2":"正确/错误",
"学生3":"正确/错误",
}

现在给定问题,请严格JSON按照格式回答。你的回答只需要包括学生答案是否正确,不要添加其他任何内容。
问题:
"""

def save_checkpoint(all_transformed_questions, current_question_numberx):
    """保存当前处理进度和结果到checkpoint文件"""
    try:
        checkpoint_data = {
            'all_transformed_questions': all_transformed_questions,
            'current_question_numberx': current_question_numberx
        }
        np.save(CHECKPOINT_FILE, checkpoint_data)
        print(f"检查点已保存到 '{CHECKPOINT_FILE}'。当前进度: {current_question_numberx}/{len(original_questions)}")
    except Exception as e:
        print(f"保存检查点时发生错误: {e}")

def load_checkpoint():
    """加载之前的处理进度和结果"""
    try:
        if os.path.exists(CHECKPOINT_FILE):
            checkpoint = np.load(CHECKPOINT_FILE, allow_pickle=True).item()
            all_transformed = checkpoint.get('all_transformed_questions', [])
            current_question_numberx = checkpoint.get('current_question_numberx', 0)
            print(f"已加载检查点文件。已处理 {current_question_numberx} 个题目,已转换 {len(all_transformed)} 个结果。")
            return all_transformed, current_question_numberx
        else:
            print("未找到检查点文件,将从头开始处理。")
            return [], 0
    except Exception as e:
        print(f"加载检查点时发生错误: {e}")
        return [], 0

def extract_pdf_text(pdf_path, save_dir=None):
    """
    提取PDF文本并保存到文件中
    
    Args:
        pdf_path (str): PDF文件路径
        save_dir (str, optional): 保存目录，默认为PDF同目录
        
    Returns:
        str: 保存的文本文件路径
    """
    # 如果未提供保存目录，使用PDF所在目录
    if save_dir is None:
        save_dir = os.path.dirname(pdf_path)
    else:
        return
    
    # 创建保存目录（如果不存在）
    os.makedirs(save_dir, exist_ok=True)
    
    # 生成保存文件名（使用PDF文件名+.json）
    pdf_filename = os.path.basename(pdf_path)
    pdf_name_without_ext = os.path.splitext(pdf_filename)[0]
    save_path = os.path.join(save_dir, f"{pdf_name_without_ext}.json")
    
    # 检查是否已经提取过文本
    if os.path.exists(save_path):
        print(f"找到已提取的文本文件: {save_path}")
        return save_path
    
    # 开始提取
    print(f"开始提取PDF文本...")
    start_time = time.time()
    
    extracted_data = {
        "pdf_path": pdf_path,
        "extraction_time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_pages": 0,
        "pages": {}
    }
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            extracted_data["total_pages"] = total_pages
            
            print(f"PDF总页数: {total_pages}")
            
            for page_num, page in enumerate(pdf.pages):
                current_page = page_num + 1  # 页码从1开始
                print(f"正在提取第 {current_page}/{total_pages} 页...")
                
                # 提取文本
                text = page.extract_text() or ""
                extracted_data["pages"][str(current_page)] = text
        
        # 保存提取结果
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, ensure_ascii=False, indent=2)
        
        end_time = time.time()
        print(f"文本提取完成！用时 {end_time - start_time:.2f} 秒")
        print(f"文本已保存至: {save_path}")
    except:
        pass


def find_text_in_saved_pdf(text_file_path, search_text):
    """
    在已保存的PDF文本中查找指定文本
    
    Args:
        text_file_path (str): 保存的文本文件路径
        search_text (str): 需要查找的文本
    
    Returns:
        list: 包含文本出现页码的列表（页码从1开始）
    """
    result_pages = []
    
    try:
        # 读取保存的文本文件
        with open(text_file_path, 'r', encoding='utf-8') as f:
            pdf_data = json.load(f)
        
        # 获取PDF总页数
        total_pages = pdf_data.get("total_pages", 0)
        pages_data = pdf_data.get("pages", {})
        
        print(f"PDF总页数: {total_pages}")
        
        # 遍历每一页
        for page_num in range(1, total_pages + 1):
            # 获取当前页面文本
            text = pages_data.get(str(page_num), "")
            
            # 检查搜索文本是否在当前页面
            if search_text in text:
                result_pages.append(page_num)
    
    except Exception as e:
        print(f"查找文本时发生错误: {e}")
    
    return result_pages

def find_mode(lst):
    count = Counter(lst)
    mode = count.most_common(1)[0][0]
    return mode

def is_chinese_char(char):
    """
    判断一个字符是否为常用的汉字。
    这里使用的是基本的CJK统一表意文字区段 (U+4E00 至 U+9FFF)。
    如果需要更广泛的汉字支持（例如扩展区A、B等），可以扩展此处的Unicode范围。
    """
    return '\u4e00' <= char <= '\u9fff'

def get_consecutive_chinese_chars(text):
    """
    从输入文本中随机查找一个由5个连续汉字组成的子字符串。
    """
    if not text or len(text) < 5:
        return ""  # 如果文本为空或长度小于10，则返回空字符串

    possible_substrings = []
    # 遍历所有可能的10字符子串的起始位置
    for i in range(len(text) - 4):  # 确保子字符串长度为10
        substring = text[i:i+5]
        # 检查子字符串中的所有字符是否都是汉字
        if all(is_chinese_char(char) for char in substring):
            possible_substrings.append(substring)
        
    return possible_substrings


# 定义 PDF 文件路径
pdf_path = r"物理学难题集萃(增订本)【舒幼生等】_part1(OCR).pdf"
json_path = r"物理学难题集萃(增订本)【舒幼生等】_part1(OCR).json"

extract_pdf_text(pdf_path, save_dir=json_path)

def call_deepseek(prompt):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": prompt}
        ],
        # temperature=0.2,
        response_format={'type': 'json_object'}
    )
    return response.choices[0].message.content


try:
    last_call = np.load(CHECKPOINT_FILE, allow_pickle=True).item()
    question_numberx = last_call['question_numberx']
    data = last_call['data']
    n_row = data.shape[0]
except:
    n_row = 0
    data = pd.DataFrame(columns=['id','问题条件','具体问题','问题数目','适合年级','题目类型','题目学科','子学科','领域类型','是否包含图片','考察知识点','易错点','思考过程/分析','解题过程','最终答案','错误解题方法','错误解法模型','错误模型','三模型打分','deepseek','qianwen','豆包','题目来源'])
                

# 主程序
try:
    # 加载题目数据
    with open(INPUT_JSON_FILE, 'r', encoding='utf-8') as f:
        original_questions = json.load(f)
except FileNotFoundError:
    print(f"错误:未找到输入文件 '{INPUT_JSON_FILE}'。")
    exit(1)

# # 加载上次处理结果
# all_transformed_questions, start_question_numberx = load_checkpoint()
# print(f"找到 {len(original_questions)} 个待处理题目。")

# if start_question_numberx > 0 and start_question_numberx < len(original_questions):
#     print(f"从索引 {start_question_numberx} 处恢复处理 (已完成 {start_question_numberx}/{len(original_questions)} 个题目)")
# else:
#     start_question_numberx = 0
#     print("从头开始处理题目。")
start_question_numberx = 0
# 处理题目
for question_numberx in range(start_question_numberx, len(original_questions)):
    problem_obj = original_questions[question_numberx]
    print(f"\n-------------------处理第 {question_numberx+1}/{len(original_questions)} 个题目 (question_number: {problem_obj.get('id')})----------------------")
    if '图' in problem_obj.get('question',''):
        print('[跳过]跳过带图题')
        continue
    if '证' in problem_obj['question']:
        print('[跳过]跳过证明题')
        
    flag = True
    count = 0
    ans_json = {}
    while flag:
        try:
            #################################### 提取五元组 #################################[question_number]、[condition]、[specific_questions]、[solution]、[final_answer]
            print(f"[提取五元组]第{count+1}次尝试")
            ans = call_deepseek(prompt_extra+'\nQURSTION:'+f"【{problem_obj['id']}】"+problem_obj['question']+'\nANSWER:'+problem_obj['answer'])
            ans_json = json.loads(ans)['result']
            flag = False
            count += 1
        except Exception as e:
            print(f'[提取五元组]Failed:{e}')
            count += 1  
            pass

    print(f"[提取五元组]Success:成功从【{problem_obj['id']}】中提取出{len(ans_json)}个五元组")

    for index,item in enumerate(ans_json): # 处理一个问题
        print(f"[处理第{index+1}个问题]正在处理第{index+1}个五元组...")

        now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        item['question_number'] = f"{item['question_number']}_{now}"
        ques_id = item['question_number']
        condition = item['condition']
        specific_questions = item['specific_questions']
        solution = item['solution']
        final_answer = item['final_answer']

        with open(f"input/{ques_id}.json", "w", encoding="utf-8") as f:
            json.dump([ans_json[index]], f, ensure_ascii=False, indent=4)

        print(f'[处理第{index+1}个问题]{ques_id}.json file saved')

        #################################### 获取三模型答案 #################################
        for llm in ['deepseek','qianwen','doubao']:

            cmd = f"cd browser-llm && node src\index.js -l {llm} -i ../input/{ques_id}.json -a zht"
            process = subprocess.Popen(
                cmd, 
                shell=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                bufsize=1,
                universal_newlines=True
            )

            output = []
            for line in iter(process.stdout.readline, ''):
                print(line, end='')
                output.append(line)
                if '全部处理完成' in line:
                    break
                # sys.stdout.flush()  # Ensure output is displayed immediately

            # Wait for the process to complete
            # return_code = process.wait()

        print(f"\n[处理第{index+1}个问题]三模型回答截图完毕")

        with open(f'browser-llm\src\outputs\qianwen\qianwen_output_{ques_id}.json', 'r', encoding='utf-8') as file_handle:
            # Load the JSON data from the file object
            qwen_ans = json.load(file_handle)['messages'][0]
        with open(f'browser-llm\src\outputs\deepseek\deepseek_output_{ques_id}.json', 'r', encoding='utf-8') as file_handle:
            # Load the JSON data from the file object
            deepseek_ans = json.load(file_handle)['messages'][0]
        with open(f'browser-llm\src\outputs\doubao\doubao_output_{ques_id}.json', 'r', encoding='utf-8') as file_handle:
            # Load the JSON data from the file object
            doubao_ans = json.load(file_handle)['messages'][0].replace('正在搜索\n','')

        #################################### 填写EXCEL表 #################################
        wrong_ans = None
        wrong_num = 0
        ds_jietu = None
        qw_jietu = None
        db_jietu = None
        cwjtff = None
        cwjfmx = None
        cwmx = ''
        shnj = None
        zxk = None
        kczsd = None
        fxgc = None
        cwjtff = None
        ycd = None

        # 判断对错
        flag = True
        count = 0
        while flag: 
            try:
                print(f"    [处理第{index+1}个问题-判断三模型答案对错]第{count+1}次尝试")
                ds_ans3 = call_deepseek(prompt_judge+condition+specific_questions+'\n正确答案:'+solution+'\学生1答案:'+qwen_ans+'\学生2答案:'+deepseek_ans+'\学生3答案:'+doubao_ans)
                ds_ans3_json = json.loads(ds_ans3)
                wrong_num = 0

                if '正确' in ds_ans3_json['学生1']:
                    qw_correct = True
                else:
                    qw_correct = False
                    wrong_num += 1
                    wrong_ans = qwen_ans
                    cwjfmx = '千问'
                    qw_jietu = f'browser-llm\src\outputs\qianwen\qianwen_screenshot_{ques_id}.png'
                    cwmx += '千问'

                if '正确' in ds_ans3_json['学生2']:
                    ds_correct = True
                else:
                    ds_correct = False
                    wrong_num += 1
                    wrong_ans = deepseek_ans
                    if not cwjfmx:
                        cwjfmx = 'ds'
                    ds_jietu = f'browser-llm\src\outputs\deepseek\deepseek_screenshot_{ques_id}.png'
                    cwmx += ',ds'

                if '正确' in ds_ans3_json['学生3']:
                    db_correct = True
                else:
                    db_correct = False
                    wrong_num += 1
                    wrong_ans = doubao_ans
                    if not cwjfmx:
                        cwjfmx = '豆包'
                    db_jietu = f'browser-llm\src\outputs\doubao\doubao_screenshot_{ques_id}.png'
                    cwmx += ',豆包'

                    
                flag = False
            except Exception as e:
                print(e)
                count += 1
                pass
        
        if qw_correct and ds_correct and db_correct: # 全部作对了,下一题
            print('[三模型都答对了，下一题]')
            continue


        ################### 提取['适合年级', '子学科']
        flag = True
        count = 0
        while flag:
            try:
                print(f"    [处理第{index+1}个问题-提取['适合年级', '子学科']]第{count+1}次尝试")
                ds_ans1 = call_deepseek(prompt_question+condition+specific_questions)
                ds_ans1_json = json.loads(ds_ans1)
                shnj = ds_ans1_json['适合年级']
                zxk = ds_ans1_json['子学科']
                flag = False
            except Exception as e:
                print(e)
                count += 1
                pass
        
        ################### 提取['考察知识点', '分析过程']
        flag = True
        count = 0
        while flag:
            try:
                print(f"    [处理第{index+1}个问题-提取['考察知识点', '分析过程']]第{count+1}次尝试")
                ds_ans2 = call_deepseek(prompt_answer+condition+specific_questions+'\n参考答案:'+solution)
                ds_ans2_json = json.loads(ds_ans2)
                kczsd = ds_ans2_json['考察知识点']
                fxgc = ds_ans2_json['分析过程']
                flag = False
            except Exception as e:
                print(e)
                count += 1
                pass
                

        ################### 提取['错误解题方法','易错点']
        flag = True
        count = 0
        while flag:
            try:
                print(f"    [处理第{index+1}个问题-提取['错误解题方法','易错点']]第{count+1}次尝试")
                ds_ans4 = call_deepseek(prompt_wrong+condition+specific_questions+'\n错误答案:'+wrong_ans+'\n正确答案:'+solution)
                ds_ans4_json = json.loads(ds_ans4)
                cwjtff = ds_ans4_json['错误解题方法']
                ycd = ds_ans4_json['易错点']
                flag = False
            except Exception as e:
                print(e)
                count += 1
                pass

        #################################### 提取['题目来源']
        possible_substrings = get_consecutive_chinese_chars(condition)
        pages = []
        if possible_substrings:
            for i in range(5):
                search_text = random.choice(possible_substrings)  # 从找到的子串中随机选择一个
                page_k = find_text_in_saved_pdf(json_path,search_text)[0]
                if page_k is not None:
                    pages.append(page_k)

        page = find_mode(pages)
            
        new_row = {
            'id': f"zht_{n_row+1:03d}",
            '问题条件': condition,
            '具体问题': specific_questions,
            '问题数目': 1,
            '适合年级': shnj,
            '题目类型': "计算题",
            '题目学科': "物理",
            '子学科': zxk,
            '领域类型': "自然科学",
            '是否包含图片': '否',
            '考察知识点': kczsd,
            '易错点': ycd,
            '思考过程/分析': fxgc,
            '解题过程': solution,
            '最终答案': final_answer,
            '错误解题方法': cwjtff,
            '错误解法模型': cwjfmx,
            '错误模型':cwmx,
            '三模型打分':wrong_num,
            'deepseek':ds_jietu,
            'qianwen':qw_jietu,
            '豆包':db_jietu,
            '题目来源':f'物理学难题集萃(增订本)【舒幼生等】_part1,第{page}页',
        }

        ################### 将数据添加到 DataFrame 中
        data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
        n_row += 1
        
        break
    
    break

        #     all_transformed_questions.extend(ans_json)
        #     flag = False

        
        #     # 每处理5个题目保存一次检查点
        #     if (question_numberx + 1) % 5 == 0 or question_numberx == len(original_questions) - 1:
        #         save_checkpoint(all_transformed_questions, question_numberx)

        # except Exception as e:
        #     count += 1
        #     if count > 5:
        #         flag = False
        #         save_checkpoint(all_transformed_questions, question_numberx-1)
        #         time.sleep(10)
            
# # 处理完成后保存最终结果
# if all_transformed_questions:
#     try:
#         with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
#             json.dump(all_transformed_questions, f, ensure_ascii=False, indent=4)
#         print(f"成功保存转换后题目至 '{OUTPUT_JSON_FILE}'。")
#     except Exception as e:
#         print(f"写入文件 '{OUTPUT_JSON_FILE}' 时发生错误:{e}")

# print(f"转换完成！共处理 {len(original_questions)} 个题目,生成 {len(all_transformed_questions)} 个转换后的题目。")

ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
data = data.map(lambda x: ILLEGAL_CHARACTERS_RE.sub('', str(x)))

now_time = time.strftime("%Y-%m-%d_%H%M%S", time.localtime())
file_path = f"三模型表/国内三模型_{now_time}.xlsx"
try:
    data.to_excel(file_path, index=False)
except Exception as e:
    print(f"写入文件时出错: {e}")


-------------------处理第 1/98 个题目 (question_number: 题 21)----------------------
[跳过]跳过带图题

-------------------处理第 2/98 个题目 (question_number: 题 22)----------------------
[提取五元组]第1次尝试
[提取五元组]Failed:Invalid \escape: line 4 column 136 (char 176)
[提取五元组]第2次尝试
[提取五元组]Failed:Invalid \escape: line 5 column 142 (char 196)
[提取五元组]第3次尝试
[提取五元组]Success:成功从【题 22】中提取出3个五元组
[处理第1个问题]正在处理第1个五元组...
[处理第1个问题]22.1_20250514230647.json file saved
[信息] 从 c:\Users\11541\Desktop\Zhet\科大数据集\0509\input\22.1_20250514230647.json 加载问题数据
[信息] 成功加载 1 个问题

🚀 开始处理 - 使用账号: zht | LLM平台: deepseek


[进度] 问题 22.1_20250514230647 (1/1) - 0% 完成
[INFO] 输出目录: c:\Users\11541\Desktop\Zhet\科大数据集\0509\browser-llm\src\outputs\deepseek
[deepseek Q22.1_20250514230647] 开始处理，尝试 #1/3
[INFO] 使用Cookie文件: cookies\zht\deepseek-state.json
[TimeDisplay] Injected live time display div (ID: playwright-live-time-display) into the page.
[deepseek Q22.1_20250514230647] 未检测到广告弹窗或关闭超时，或弹窗不存在。
[deepseek Q22.1_20250514230647] 输入框可见
[deepseek Q22.1_20250

In [38]:
ans = call_deepseek(prompt_extra+'\nQURSTION:'+f"【{problem_obj['id']}】"+problem_obj['question']+'\nANSWER:'+problem_obj['answer'])

In [ ]:
print(prompt_extra+'\nQURSTION:'+f"【{problem_obj['id']}】"+problem_obj['question']+'\nANSWER:'+problem_obj['answer'])

In [41]:
json.loads(ans)['result']

[{'question_number': '22.1',
  'condition': '某空调器按可逆卡诺循环运转，其中的作功装置连续工作时所提供的功率为 $P_{0}$ 。夏天，室外温度为恒定的 $T_{1}$ ，启动空调器连续工作，最后可将室温降至恒定的 $T_{2}$ 。室外通过热传导在单位时间内向室内传输的热量正比于 $\\left(T_{1}-T_{2}\\right)$ （牛顿冷却定律），比例系数为 $A$ 。',
  'specific_questions': '试用 $T_{1}, P_{0}$ 和 $A$ 来表示 $T_{2}$ 。',
  'solution': '为了保持室温恒定，空调器从室内吸收的热量应等于室外向室内通过热传导传输的热量。单位时间从室内吸热 $Q_{2}$ ，向室外放热 $Q_{1}$ ，故 $Q_{1}=Q_{2}+P$ 。因空调器作可逆卡诺循环，有 $\\frac{Q_{1}}{T_{1}}=\\frac{Q_{2}}{T_{2}}$ 。由此得出 $Q_{2}=\\frac{T_{2}}{T_{1}-T_{2}} P$ 。同时，单位时间内室外向室内通过热传导传输的热量为 $Q=A\\left(T_{1}-T_{2}\\right)$ 。为了保持室温恒定，应有 $Q=Q_{2}$ ，即 $A\\left(T_{1}-T_{2}\\right)=\\frac{T_{2}}{T_{1}-T_{2}} P$ 。解这个方程，得到 $T_{2}=T_{1}+\\frac{1}{2}\\left[\\frac{P_{0}}{A}-\\sqrt{\\left(\\frac{P_{0}}{A}\\right)^{2}+\\frac{4 P_{0}}{A} T_{1}}\\right]$ 。',
  'final_answer': '$$T_{1}+\\frac{1}{2}\\left[\\frac{P_{0}}{A}-\\sqrt{\\left(\\frac{P_{0}}{A}\\right)^{2}+\\frac{4 P_{0}}{A} T_{1}}\\right]$$'},
 {'question_number': '22.2',
  'condition': '某空调器按可逆卡诺循环运转，其中的作功装置连续工作时所提供的功率为 $P

In [33]:
ds_ans4 = call_deepseek(prompt_wrong+condition+specific_questions+'\n错误答案:'+wrong_ans_+'\n正确答案:'+solution)
ds_ans4_json = json.loads(ds_ans4)
ds_ans4_json

{'错误解题方法': '答题过程：空调作为可逆卡诺制冷机，其制冷系数为 ε=T1\u200b−T2\u200bT2\u200b\u200b。制冷功率 Qc\u200b=εP0\u200b=T1\u200b−T2\u200bT2\u200bP0\u200b\u200b。热传导输入的热量率为 A(T1\u200b−T2\u200b)。稳态时两者平衡：@Wrong_Step:reason@错误地将制冷功率和热传导输入的热量率直接相等，忽略了正确的热平衡条件 T1\u200b−T2\u200bT2\u200bP0\u200b\u200b=A(T1\u200b−T2\u200b)整理得：A(T1\u200b−T2\u200b)2=T2\u200bP0\u200b展开并整理为二次方程：AT22\u200b−(2AT1\u200b+P0\u200b)T2\u200b+AT12\u200b=0解得：T2\u200b=2A2AT1\u200b+P0\u200b−4AP0\u200bT1\u200b+P02\u200b\u200b\u200b最终答案：T2\u200b=2A2AT1\u200b+P0\u200b−4AP0\u200bT1\u200b+P02\u200b\u200b\u200b回答完毕',
 '易错点': '错误地将制冷功率和热传导输入的热量率直接相等，忽略了正确的热平衡条件，以及在解二次方程时可能出现的计算错误和忽略不合理解的情况。'}

In [4]:
from openai import OpenAI
import json
import os
import time
import numpy as np
import datetime
import subprocess
from dotenv import load_dotenv
import fitz  # PyMuPDF
import re
import pandas as pd

# 如果存在 .env 文件,从中加载环境变量
load_dotenv()
# 配置代理（如需要）
# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# client = OpenAI(
#     api_key=os.getenv("openai_api"),
# )
# MODEL = "gpt-4o-mini"
# INPUT_JSON_FILE = "questions_extracted.json"
# OUTPUT_JSON_FILE = "questions_transformed_gpt.json"
# CHECKPOINT_FILE = "last_call_gpt.npy"  # 添加检查点文件名

client = OpenAI(
    api_key=os.getenv("deepseek_api"),
    base_url="https://api.deepseek.com",
)
MODEL = "deepseek-chat"

INPUT_JSON_FILE = "questions_extracted.json"
OUTPUT_JSON_FILE = "questions_transformed_ds.json"
CHECKPOINT_FILE = "last_call_ds.npy"  # 添加检查点文件名

# --- 提示词 ---

prompt_extra = """
给定一个问题的原始题干和原始答案,按照以下规则提取出[question_number]、[condition]、[specific_questions]、[solution]、[final_answer],并返回一个JSON列表。

其中:
"question_number": "字符串,题号"
"condition": "原题的题干,直接复制原题的Latex内容,然后参考转换规则修改",
"specific_questions": "原题的设问,直接复制原题的Latex内容,然后参考转换规则修改",
"solution": "子问的逐步solution,改写为独立,不包含图示,且不直接引用其他部分结论。如需前部结果,应作为已知条件说明",
"final_answer": "一个数值或公式,不要任何汉字、条件、单位, 不要出现 '=','\n','\box'"


转换规则:
1. 输出格式: 输出必须是一个JSON格式{"result":[字典列表]}, 每个字典必须是latex格式,确保能用latex编译器编译通过
2. 单个子问对应一个 JSON 对象: 若原题包含多个子问题(如 1., 2., a., b.),拆分为多个对象,question_number 用原 question_number.x。
3. 子问题独立: 每个对象必须尽量自包含。"condition"和"solution"避免出现"由第 1 部分得出"等表述,若需前部结果,请将结果写入该部分的"condition"。
4. 无图示引用: 删除"condition""specific_questions""solution"中所有图示或图片引用,如"如图所示""图 x-x-x""见图"等。
5. 跳过证明题: 若"specific_questions"为证明题,则跳过该子问;若整题仅证明题,则输出 `[]`。
6. 转换选填题: 若"specific_questions"为选择题或填空题,需转换为计算题并给出数值或公式;不可行则跳过。
7. 内容完整: 保留题目核心物理概念和数值,改写不改变实质。
8. 结构: 严格包含"question_number"、"condition"、"specific_questions"、"solution"、"final_answer"五个字段。
9. 需要删除的内容 :【多余的补位符号(*#?-)】【题目序号】【如图xx所示】【某个大学】【某个省份】,"final_answer"里的汉字、单位、条件
10. 需要核对的内容 : 核对原题目和答案汉字的正确性:人射 vs 入射、代人 vs 代入、收玫 vs 收敛;latex代码的正确性:$\\overrightarrow{{AB}}$ vs $\\overline{{AB}}$
11. 需要修改的格式 : \\[替换为 换行$$,\\]替换为 $$换行,\\(替换为$,\\)替换为$;

EXAMPLE INPUT:
QURSTION:

        

ANSWER:
解答 (1) 星球的质量

$$
M=\int_{0}^{R} \sqrt{-g_{r r} \rho}(r) \mathrm{d} r \int_{0}^{\pi} \sqrt{-g_{\theta \theta}} \mathrm{d} \theta \int_{0}^{2 \pi} \sqrt{-g_{\varphi \varphi}} \mathrm{d} \varphi
$$

所以

$$
M=4 \pi \int_{0}^{R} \rho(r) \frac{1}{\sqrt{1-r^{2} Y(r)}} r^{2} \mathrm{~d} r
$$

类似质量,星球中的总强子数,

$$
N=4 \pi \int_{0}^{R} n(r) \frac{r^{2} \mathrm{~d} r}{\sqrt{1-r^{2} Y(r)}}
$$

星球中心到星体表面的红移

$$
\frac{\lambda_{2}}{\lambda_{1}}=\left[\frac{g_{00}\left(x_{1}\right)}{g_{00}\left(x_{2}\right)}\right]^{1 / 2}=\mathrm{e}^{\Phi(r)-\Phi(0)}
$$

星体表面到远距离观察者引力导致红移

$$
\frac{\lambda_{3}}{\lambda_{2}}=\sqrt{1-\frac{2 G M}{c^{2} R}}
$$

远距离的观察者所测得的在星球中心的红移

$$
Z=\frac{\Delta \lambda}{\lambda}=\frac{\lambda_{3}}{\lambda_{1}}-1=\frac{\lambda_{2}}{\lambda_{1}} \frac{\lambda_{3}}{\lambda_{2}}-1=\sqrt{1-\frac{2 G M}{c^{2} R}} \mathrm{e}^{\Phi(r)-\Phi(0)}-1
$$

(2) 静力平衡的 Euler 方程为

$$
-\frac{\partial p}{\partial x^{\mu}}=(p+\rho) \frac{\partial}{\partial x^{\mu}} \ln \left(g_{00}\right)^{1 / 2}
$$

所以由本题的条件直接可得

$$
-\frac{\mathrm{d} p(r)}{\mathrm{d} r}=[p(r)+\rho(r)] \frac{\mathrm{d}}{\mathrm{~d} r} \ln \left[\mathrm{e}^{2 \Phi}\right]^{1 / 2}=[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}
$$

EXAMPLE JSON OUTPUT:
{"result":[{
"question_number": "7.7.1" ,
"condition":"一个静态球对称相对论星球的线元可写成

$$
\mathrm{d} s^{2}=e^{2 \Phi} \mathrm{~d} t^{2}-\frac{\mathrm{d} r^{2}}{1-r^{2} Y(r)}-r^{2} \mathrm{~d} \theta^{2}-r^{2} \sin ^{2} \theta \mathrm{~d} \varphi^{2}
$$

其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.",
"specific_questions":"用该线元中的二个已知函数 $\Phi$ 和 $Y$ 给出此星球的质量以及远距离的观察者所测得的在星球中心的红移, 用已知函数 $\Phi 、 Y$ 和 $n$ 表示该星球中的总强子数.",
            cwjtff = ''
            cwjfmx = ''
            cwmx = ''
            shnj = ''
            zxk = ''
            kczsd = ''
            fxgc = ''


所以

$$
M=4 \pi \int_{0}^{R} \rho(r) \frac{1}{\sqrt{1-r^{2} Y(r)}} r^{2} \mathrm{~d} r
$$

类似质量,星球中的总强子数,

$$
N=4 \pi \int_{0}^{R} n(r) \frac{r^{2} \mathrm{~d} r}{\sqrt{1-r^{2} Y(r)}}
$$

星球中心到星体表面的红移

$$
\frac{\lambda_{2}}{\lambda_{1}}=\left[\frac{g_{00}\left(x_{1}\right)}{g_{00}\left(x_{2}\right)}\right]^{1 / 2}=\mathrm{e}^{\Phi(r)-\Phi(0)}
$$

星体表面到远距离观察者引力导致红移

$$
\frac{\lambda_{3}}{\lambda_{2}}=\sqrt{1-\frac{2 G M}{c^{2} R}}
$$

远距离的观察者所测得的在星球中心的红移

$$


\mathrm{d} s^{2}=e^{2 \Phi} \mathrm{~d} t^{2}-\frac{\mathrm{d} r^{2}}{1-r^{2} Y(r)}-r^{2} \mathrm{~d} \theta^{2}-r^{2} \sin ^{2} \theta \mathrm{~d} \varphi^{2}
$$

其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.",
"specific_questions": "用已知函数给出静力平衡条件.",
"solution": "静力平衡的 Euler 方程为

$$
-\frac{\partial p}{\partial x^{\mu}}=(p+\rho) \frac{\partial}{\partial x^{\mu}} \ln \left(g_{00}\right)^{1 / 2}
$$

所以由本题的条件直接可得

$$
-\frac{\mathrm{d} p(r)}{\mathrm{d} r}=[p(r)+\rho(r)] \frac{\mathrm{d}}{\mathrm{~d} r} \ln \left[\mathrm{e}^{2 \Phi}\right]^{1 / 2}=[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}
$$",
"final_answer":$$[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}$$}
]}

现在你已经知道了转换规则,请按照转换规则将给定的问题转换为JSON列表。
"""

prompt_question = """
给定一个物理问题, 请给出该问题的 ['适合年级', '子学科']

注意:
1. 适合年级只能是大学的题目, 分析题目适合大学几年级学生做, 直接写[大一/大二/大三/大四]
2. 子学科是物理具体的学科分支,如力学、电磁学等

EXAMPLE OUTPUT:
{
"适合年级":"大一"
"子学科":"电磁学"
}

现在给定问题,请严格JSON式回答。你的回答只需要包括适合年级和子学科,不要添加其他任何内容。
问题:
"""

prompt_answer = """
给定一个物理问题及参考答案, 请给出该问题的 ['考察知识点', '分析过程']

注意:
1. 考察知识点需要按照['知识点1','知识点2',...]格式列出题目涉及的主要知识点
2. 分析过程需要凝练参考答案的分析过程,用一段话简洁地总结参考答案的思考过程
3. 分析过程应该尽可能凝练简洁,尽可能不包含公式, 如果一定要包含某条公式, 需要给出公式的LaTex格式

EXAMPLE JSON OUTPUT:
{
"考察知识点":['库伦定律', '受力分析', '共力点']
"分析过程": "通过分析可移动小环在倾斜杆上最高点与最低点的受力情况,利用沿杆方向的力平衡条件（结合库仑定律和静摩擦力公式）建立方程,分别求解出能停住的上下临界位置的$y$坐标,从而确定其范围。"
}

现在给定问题,请严格JSON按照格式回答。你的回答只需要包括考察知识点和分析过程,不要添加其他任何内容。
问题:
"""

prompt_wrong = """
给定一个物理问题、正确答案以及错误答案, 请给出该问题的 ['错误解题方法','易错点']

注意:
1. 【错误解题方法】即错误答案,需要对比正确答案与错误答案的步骤,在错误答案中标注错误位置,标注格式和方法如下:【@Wrong_Step:reason@易错点】
2. 【易错点】需要用一段话总结易错的地方,需要包含【@Wrong_Step:reason@易错点】中的全部易错点,以及其他的易错点(即使错误答案没有犯这个错)

EXAMPLE OUTPUT:
{
"错误解题方法":"...该级数为复数对数级数:$\sum_{n-1}^{\infty}\dfrac{o^{inx}}{n} = -\ln(1-e^{ix})$(当$e^{ix}=1$且$x \neq 2k\pi$时收敛)。
代入得:@rong_Step:reason@误用发散级数的导数积分
$S(x)=\text{Im} \left[-\ln(1-o{ix})\right]$...
"
"易错点":"...,误用发散级数的导数积分,..."
}

现在给定问题,请严格JSON按照格式回答。你的回答只需要包括错误解题方法和易错点,不要添加其他任何内容。
问题:
"""

prompt_judge = """
给定一个物理问题、正确答案以及三个学生答案, 请判断学生答案是否正确。

INPUT:
物理问题:...
正确答案:...
学生1答案:...
学生2答案:...
学生3答案:...


EXAMPLE OUTPUT:
{
"学生1":"正确/错误",
"学生2":"正确/错误",
"学生3":"正确/错误",
}

现在给定问题,请严格JSON按照格式回答。你的回答只需要包括学生答案是否正确,不要添加其他任何内容。
问题:
"""

def save_checkpoint(all_transformed_questions, current_question_numberx):
    """保存当前处理进度和结果到checkpoint文件"""
    try:
        checkpoint_data = {
            'all_transformed_questions': all_transformed_questions,
            'current_question_numberx': current_question_numberx
        }
        np.save(CHECKPOINT_FILE, checkpoint_data)
        print(f"检查点已保存到 '{CHECKPOINT_FILE}'。当前进度: {current_question_numberx}/{len(original_questions)}")
    except Exception as e:
        print(f"保存检查点时发生错误: {e}")

def load_checkpoint():
    """加载之前的处理进度和结果"""
    try:
        if os.path.exists(CHECKPOINT_FILE):
            checkpoint = np.load(CHECKPOINT_FILE, allow_pickle=True).item()
            all_transformed = checkpoint.get('all_transformed_questions', [])
            current_question_numberx = checkpoint.get('current_question_numberx', 0)
            print(f"已加载检查点文件。已处理 {current_question_numberx} 个题目,已转换 {len(all_transformed)} 个结果。")
            return all_transformed, current_question_numberx
        else:
            print("未找到检查点文件,将从头开始处理。")
            return [], 0
    except Exception as e:
        print(f"加载检查点时发生错误: {e}")
        return [], 0



# 定义 PDF 文件路径
pdf_path = r"物理学难题集萃(增订本)【舒幼生等】_part1(OCR).pdf"

doc = fitz.open(pdf_path)
def find_page_number(doc, search_text):
    # 遍历每一页,查找内容
    for page_num in range(len(doc)):
        page = doc[page_num]  # 获取当前页
        text_instances = page.search_for(search_text)  # 搜索内容

        # 如果找到内容,打印页码
        if text_instances:
            return page_num + 1

def call_deepseek(prompt):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": prompt}
        ],
        # temperature=0.2,
        response_format={'type': 'json_object'}
    )
    return response.choices[0].message.content



id = '22.1_20250513220311'
index = 0

with open(f'input/{id}.json', 'r', encoding='utf-8') as file_handle:
    item = json.load(file_handle)[0]


ques_id = item['question_number']
condition = item['condition']
specific_questions = item['specific_questions']
solution = item['solution']
final_answer = item['final_answer']



with open(f'browser-llm\src\outputs\qianwen\qianwen_output_{id}.json', 'r', encoding='utf-8') as file_handle:
    # Load the JSON data from the file object
    qwen_ans = json.load(file_handle)['messages'][0]
    # qwen_ans = 'i don\'t know'
with open(f'browser-llm\src\outputs\deepseek\deepseek_output_{id}.json', 'r', encoding='utf-8') as file_handle:
    # Load the JSON data from the file object
    deepseek_ans = json.load(file_handle)['messages'][0]
with open(f'browser-llm\src\outputs\doubao\doubao_output_{id}.json', 'r', encoding='utf-8') as file_handle:
    # Load the JSON data from the file object
    doubao_ans = json.load(file_handle)['messages'][0]

#################################### 填写EXCEL表 #################################
wrong_ans = None
wrong_num = 0
ds_jietu = None
qw_jietu = None
db_jietu = None
cwjtff = None
cwjfmx = None
cwmx = ''
shnj = None
zxk = None
kczsd = None
fxgc = None
cwjtff = None
ycd = None

# 判断对错
flag = True
count = 0

print(f"    [处理第{index+1}个问题-判断三模型答案对错]第{count+1}次尝试")
ds_ans3 = call_deepseek(prompt_judge+condition+specific_questions+'\n正确答案:'+solution+'\学生1答案:'+qwen_ans+'\学生2答案:'+deepseek_ans+'\学生3答案:'+doubao_ans)
ds_ans3_json = json.loads(ds_ans3)
wrong_num = 0

if '正确' in ds_ans3_json['学生1']:
    qw_correct = True
else:
    qw_correct = False
    wrong_num += 1
    wrong_ans = qwen_ans
    cwjfmx = '千问'
    qw_jietu = f'browser-llm\src\outputs\qianwen\qianwen_screenshot_{id}.png'
    cwmx += '千问'

if '正确' in ds_ans3_json['学生2']:
    ds_correct = True
else:
    ds_correct = False
    wrong_num += 1
    wrong_ans = deepseek_ans
    if not cwjfmx:
        cwjfmx = 'ds'
    ds_jietu = f'browser-llm\src\outputs\deepseek\deepseek_screenshot_{id}.png'
    cwmx += ',ds'

if '正确' in ds_ans3_json['学生3']:
    db_correct = True
else:
    db_correct = False
    wrong_num += 1
    wrong_ans = doubao_ans
    if not cwjfmx:
        cwjfmx = '豆包'
    db_jietu = f'browser-llm\src\outputs\doubao\doubao_screenshot_{id}.png'
    cwmx += ',豆包'



# if qw_correct and ds_correct and db_correct: # 全部作对了,下一题
#     print('all right')
#     exit()


################### 提取['适合年级', '子学科']
# flag = True
# count = 0
# while flag:
#     try:
#         print(f"    [处理第{index+1}个问题-提取['适合年级', '子学科']]第{count+1}次尝试")
#         ds_ans1 = call_deepseek(prompt_question+condition+specific_questions)
#         ds_ans1_json = json.loads(ds_ans1)
#         shnj = ds_ans1_json['适合年级']
#         zxk = ds_ans1_json['子学科']
#         flag = False
#     except:
#         count += 1
#         pass

# ################### 提取['考察知识点', '分析过程']
# flag = True
# count = 0

# print(f"    [处理第{index+1}个问题-提取['考察知识点', '分析过程']]第{count+1}次尝试")
# ds_ans2 = call_deepseek(prompt_answer+condition+specific_questions+'\n参考答案:'+solution)
# ds_ans2_json = json.loads(ds_ans2)
# kczsd = ds_ans2_json['考察知识点']
# fxgc = ds_ans2_json['分析过程']
# flag = False

        

# ################### 提取['错误解题方法','易错点']
# flag = True
# count = 0

print(f"    [处理第{index+1}个问题-提取['错误解题方法','易错点']]第{count+1}次尝试")
ds_ans4 = call_deepseek(prompt_wrong+condition+specific_questions+'\n错误答案:'+wrong_ans+'\n正确答案:'+solution)
ds_ans4_json = json.loads(ds_ans4)
cwjtff = ds_ans4_json['错误解题方法']
ycd = ds_ans4_json['易错点']
flag = False

#################################### 提取['题目来源']
# search_text = condition[:10]
# page = find_page_number(doc, search_text)
# if not page:
#     search_text = f"题{id.split('_')[0]}"
#     page = find_page_number(doc, search_text)
    
# page
# new_row = {
#     'id': f"zht_{n_row+1:03d}",
#     '问题条件': condition,
#     '具体问题': specific_questions,
#     '问题数目': 1,
#     '适合年级': shnj,
#     '题目类型': "计算题",
#     '题目学科': "物理",
#     '子学科': zxk,
#     '领域类型': "自然科学",
#     '是否包含图片': '否',
#     '考察知识点': kczsd,
#     '易错点': ycd,
#     '思考过程/分析': fxgc,
#     '解题过程': solution,
#     '最终答案': final_answer,
#     '错误解题方法': cwjtff,
#     '错误解法模型': cwjfmx,
#     '错误模型':cwmx,
#     '三模型打分':wrong_num,
#     'deepseek':ds_jietu,
#     'qianwen':qw_jietu,
#     '豆包':db_jietu,
#     '题目来源':f'物理学难题集萃(增订本)【舒幼生等】_part1,第{page}页',
# }

# ################### 将数据添加到 DataFrame 中
# data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

    [处理第1个问题-判断三模型答案对错]第1次尝试
    [处理第1个问题-提取['错误解题方法','易错点']]第1次尝试


JSONDecodeError: Expecting value: line 2 column 10 (char 11)

In [8]:
print(wrong_ans)

嗯，我现在要解决这个问题，关于空调器按照可逆卡诺循环工作，然后求室温T2的表达式，用T1、P0和A来表示。让我仔细看看题目条件。
首先，题目说空调器是按可逆卡诺循环运转的，也就是它是一个理想热泵。卡诺循环的效率应该和热力学效率有关。当空调在夏天工作时，它应该是在从室内吸热，然后排到室外，对吧？不过这里可能要注意，因为题目里说室外温度是恒定的T1，而室内最终温度是T2。所以空调的作用应该是将室内的热量转移到室外，从而降低室内温度，直到达到平衡状态。
题目还提到，当空调连续工作时，作功装置提供的功率是P0。功率应该就是单位时间内外界对系统做的功，也就是单位时间的输入功。而根据卡诺循环的特性，热泵的性能系数（COP）应该是COP = T2/(T1 - T2)，因为当热泵从低温环境（T2）吸热，排到高温环境（T1）时，COP的公式是这样的。或者说，这里可能需要再仔细确认一下。
不过，卡诺热泵的COP公式是COP_H = T2/(T1 - T2)，对吗？因为热泵的供热系数是Q_H/W，而Q_H是传递到高温侧的热量，这里可能要看具体的方向。或者，当空调在制冷模式下，也就是从室内吸热，排到室外，这时候制冷系数COP_R应该是Q_L/W = T2/(T1 - T2)？或者反过来？
可能我需要再仔细回忆一下卡诺循环的公式。卡诺热泵的供热系数（用于制热）是COP_H = T_hot/(T_hot - T_cold)，而制冷系数COP_R = T_cold/(T_hot - T_cold)。在这里，当空调在制冷模式下，也就是把热量从室内（低温T2）转移到室外（高温T1），所以制冷系数COP_R应该是 T2/(T1 - T2)，而此时，单位时间的制冷量Q_L = COP_R * P0，因为功率P0是单位时间的功W。
不过题目中可能需要考虑的是，空调在制冷时，单位时间从室内取出的热量Q_L等于COP_R乘以输入的功率P0。因此，Q_L = (T2/(T1-T2)) * P0。
不过，除了空调的制冷作用外，题目还提到室外通过热传导在单位时间内向室内传输的热量正比于(T1 - T2)，比例系数A。也就是说，热传导带来的热量是Q_cond = A*(T1 - T2)。这个热量是外界传入室内的，所以当空调在制冷时，它需要把这部分热量以及可能的其他热量排出，才能维持室温T2的稳定。
当系统达到稳定状态

In [9]:
cwjfmx

'千问'

In [ ]:
# gemini
import json
import os
import numpy as np
from dotenv import load_dotenv
import google.generativeai as genai

# node src/index.js -l deepseek -i "input.json" -a zht

# --- 配置 ---
# 如果存在 .env 文件,从中加载环境变量
load_dotenv()
# 配置代理（如需要）
GOOGLE_API_KEY = os.getenv("gemini_api")
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
else:
    print("错误:未设置 GOOGLE_API_KEY 环境变量。请设置该变量以使用 Gemini API。")

GEMINI_MODEL_NAME = "gemini-2.5-flash-preview-04-17"

INPUT_JSON_FILE = "questions_extracted.json"
OUTPUT_JSON_FILE = "questions_transformed_gemini.json"

CHECKPOINT_FILE = "last_call_gemini.npy"  # 添加检查点文件名

# --- 提示词 ---
SYSTEM_PROMPT = """
你是一个专精于转换中文教材物理题目的专家 AI 助手。
你的任务是将每个给定的问题重新格式化为一个或多个 JSON 对象。每个 JSON 对象必须表示一个独立的题目部分,并遵循指定的结构。
严格遵守所有转换规则。
"""

USER_PROMPT_TEMPLATE = """
给定一个问题的原始题干和原始答案,按照以下规则提取出[question_number]、[condition]、[specific_questions]、[solution]、[final_answer],并返回一个JSON列表。

其中:
"question_number": "字符串,题号"
"condition": "字符串(也就是题的题干,直接复制原题的Latex内容,然后参考转换规则修改)",
"specific_questions": "字符串(也就是题的题干,直接复制原题的Latex内容,然后参考转换规则修改)",
"solution": "字符串(该子问的逐步solution,改写为独立,不包含图示,且不直接引用其他部分结论。如需前部结果,应作为已知条件说明)",
"final_answer": "字符串(只要一个数值或公式内容,不要任何汉字、条件、单位,不要出现等式、等号、\n、\box 等一切其他内容)"


转换规则:
1. 输出格式: 输出必须是一个JSON格式{"result":[字典列表]}, 每个字典必须是latex格式,确保能用latex编译器编译通过
2. 单个子问对应一个 JSON 对象: 若原题包含多个子问题(如 1., 2., a., b.),拆分为多个对象,question_number 用原 question_number.x。
3. 子问题独立: 每个对象必须尽量自包含。"condition"和"solution"避免出现"由第 1 部分得出"等表述,若需前部结果,请将结果写入该部分的"condition"。
4. 无图示引用: 删除"condition""specific_questions""solution"中所有图示或图片引用,如"如图所示""图 x-x-x""见图"等。
5. 跳过证明题: 若"specific_questions"为证明题,则跳过该子问;若整题仅证明题,则输出 `[]`。
6. 转换选填题: 若"specific_questions"为选择题或填空题,需转换为计算题并给出数值或公式;不可行则跳过。
7. 内容完整: 保留题目核心物理概念和数值,改写不改变实质。
8. 结构: 严格包含"question_number"、"condition"、"specific_questions"、"solution"、"final_answer"五个字段。
9. 需要删除的内容 :【多余的补位符号(*#?-)】【题目序号】【如图xx所示】【某个大学】【某个省份】
10. 需要核对的内容 : 核对原题目和答案汉字的正确性:人射 vs 入射、代人 vs 代入、收玫 vs 收敛;latex代码的正确性:$\\overrightarrow{{AB}}$ vs $\\overline{{AB}}$
11. 需要修改的格式 : \\[替换为 换行$$,\\]替换为 $$换行,\\(替换为$,\\)替换为$;

EXAMPLE INPUT:
QURSTION:
题 7.7 一个静态球对称相对论星球的线元可写成$$\mathrm{{d}} s^{{2}}=e^{{2 \Phi}} \mathrm{{d}} t^{{2}}-\frac{{\mathrm{{d}} r^{{2}}}}{{1-r^{{2}} Y(r)}}-r^{{2}} \mathrm{{d}} \theta^{{2}}-r^{{2}} \sin^{{2}} \theta \mathrm{{d}} \varphi^{{2}}$$其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.
(1) 用该线元中的二个已知函数 $\Phi$ 和 $Y$ 给出此星球的质量以及远距离的观察者所测得的在星球中心的红移, 用已知函数 $\Phi 、 Y$ 和 $n$ 表示该星球中的总强子数.
(2) 用已知函数给出静力平衡条件.

ANSWER:
解答 (1) 星球的质量

$$
M=\int_{0}^{R} \sqrt{-g_{r r} \rho}(r) \mathrm{d} r \int_{0}^{\pi} \sqrt{-g_{\theta \theta}} \mathrm{d} \theta \int_{0}^{2 \pi} \sqrt{-g_{\varphi \varphi}} \mathrm{d} \varphi
$$

所以

$$
M=4 \pi \int_{0}^{R} \rho(r) \frac{1}{\sqrt{1-r^{2} Y(r)}} r^{2} \mathrm{~d} r
$$

类似质量,星球中的总强子数,

$$
N=4 \pi \int_{0}^{R} n(r) \frac{r^{2} \mathrm{~d} r}{\sqrt{1-r^{2} Y(r)}}
$$

星球中心到星体表面的红移

$$
\frac{\lambda_{2}}{\lambda_{1}}=\left[\frac{g_{00}\left(x_{1}\right)}{g_{00}\left(x_{2}\right)}\right]^{1 / 2}=\mathrm{e}^{\Phi(r)-\Phi(0)}
$$

星体表面到远距离观察者引力导致红移

$$
\frac{\lambda_{3}}{\lambda_{2}}=\sqrt{1-\frac{2 G M}{c^{2} R}}
$$

远距离的观察者所测得的在星球中心的红移

$$
Z=\frac{\Delta \lambda}{\lambda}=\frac{\lambda_{3}}{\lambda_{1}}-1=\frac{\lambda_{2}}{\lambda_{1}} \frac{\lambda_{3}}{\lambda_{2}}-1=\sqrt{1-\frac{2 G M}{c^{2} R}} \mathrm{e}^{\Phi(r)-\Phi(0)}-1
$$

(2) 静力平衡的 Euler 方程为

$$
-\frac{\partial p}{\partial x^{\mu}}=(p+\rho) \frac{\partial}{\partial x^{\mu}} \ln \left(g_{00}\right)^{1 / 2}
$$

所以由本题的条件直接可得

$$
-\frac{\mathrm{d} p(r)}{\mathrm{d} r}=[p(r)+\rho(r)] \frac{\mathrm{d}}{\mathrm{~d} r} \ln \left[\mathrm{e}^{2 \Phi}\right]^{1 / 2}=[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}
$$

EXAMPLE JSON OUTPUT:
{"result":[{
"question_number": "7.7.1" ,
"condition":"一个静态球对称相对论星球的线元可写成

$$
\mathrm{d} s^{2}=e^{2 \Phi} \mathrm{~d} t^{2}-\frac{\mathrm{d} r^{2}}{1-r^{2} Y(r)}-r^{2} \mathrm{~d} \theta^{2}-r^{2} \sin ^{2} \theta \mathrm{~d} \varphi^{2}
$$

其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.",
"specific_questions":"用该线元中的二个已知函数 $\Phi$ 和 $Y$ 给出此星球的质量以及远距离的观察者所测得的在星球中心的红移, 用已知函数 $\Phi 、 Y$ 和 $n$ 表示该星球中的总强子数.",
"solution":"星球的质量

$$
M=\int_{0}^{R} \sqrt{-g_{r r} \rho}(r) \mathrm{d} r \int_{0}^{\pi} \sqrt{-g_{\theta \theta}} \mathrm{d} \theta \int_{0}^{2 \pi} \sqrt{-g_{\varphi \varphi}} \mathrm{d} \varphi
$$

所以

$$
M=4 \pi \int_{0}^{R} \rho(r) \frac{1}{\sqrt{1-r^{2} Y(r)}} r^{2} \mathrm{~d} r
$$

类似质量,星球中的总强子数,

$$
N=4 \pi \int_{0}^{R} n(r) \frac{r^{2} \mathrm{~d} r}{\sqrt{1-r^{2} Y(r)}}
$$

星球中心到星体表面的红移

$$
\frac{\lambda_{2}}{\lambda_{1}}=\left[\frac{g_{00}\left(x_{1}\right)}{g_{00}\left(x_{2}\right)}\right]^{1 / 2}=\mathrm{e}^{\Phi(r)-\Phi(0)}
$$

星体表面到远距离观察者引力导致红移

$$
\frac{\lambda_{3}}{\lambda_{2}}=\sqrt{1-\frac{2 G M}{c^{2} R}}
$$

远距离的观察者所测得的在星球中心的红移

$$
Z=\frac{\Delta \lambda}{\lambda}=\frac{\lambda_{3}}{\lambda_{1}}-1=\frac{\lambda_{2}}{\lambda_{1}} \frac{\lambda_{3}}{\lambda_{2}}-1=\sqrt{1-\frac{2 G M}{c^{2} R}} \mathrm{e}^{\Phi(r)-\Phi(0)}-1
$$",
"final_answer":"$$\sqrt{1-\frac{2 G M}{c^{2} R}} \mathrm{e}^{\Phi(r)-\Phi(0)}-1$$"},
{"question_number": "7.7.2" ,
"condition": "一个静态球对称相对论星球的线元可写成

$$
\mathrm{d} s^{2}=e^{2 \Phi} \mathrm{~d} t^{2}-\frac{\mathrm{d} r^{2}}{1-r^{2} Y(r)}-r^{2} \mathrm{~d} \theta^{2}-r^{2} \sin ^{2} \theta \mathrm{~d} \varphi^{2}
$$

其中 $\Phi$ 和 $Y$ 为 $r$ 的函数. 固有质量密度为 $\rho(r)$, 压强为 $p(r)$, 固有数密度为 $n(r)$.",
"specific_questions": "用已知函数给出静力平衡条件.",
"solution": "静力平衡的 Euler 方程为

$$
-\frac{\partial p}{\partial x^{\mu}}=(p+\rho) \frac{\partial}{\partial x^{\mu}} \ln \left(g_{00}\right)^{1 / 2}
$$

所以由本题的条件直接可得

$$
-\frac{\mathrm{d} p(r)}{\mathrm{d} r}=[p(r)+\rho(r)] \frac{\mathrm{d}}{\mathrm{~d} r} \ln \left[\mathrm{e}^{2 \Phi}\right]^{1 / 2}=[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}
$$",
"final_answer":$$[p(r)+\rho(r)] \frac{\mathrm{d} \Phi(r)}{\mathrm{d} r}$$}
]}

现在你已经知道了转换规则,请按照转换规则将给定的问题转换为JSON列表。
"""

def call_gpt_for_transformation(problem_obj):
    """
    调用 GPT API 模型转换单个题目对象。
    problem_obj 是包含 "question_number"、"question"、"answer" 的字典。
    返回转换后的题目字典列表,若跳过则返回 []。
    """
    original_question_number = problem_obj.get("question_number", "")
    original_question = original_question_number + ' ' + problem_obj.get("question", "")
    original_answer = problem_obj.get("answer", "")

    original_question_escaped = original_question.replace("`", "\\`")
    original_answer_escaped = original_answer.replace("`", "\\`")

    prompt = USER_PROMPT_TEMPLATE+f"QURSTION:\n{original_question_escaped}\nANSWER:\n{original_answer_escaped}"

    response_content = None # Initialize for a broader scope
    try:
        model = genai.GenerativeModel(
            GEMINI_MODEL_NAME,
            system_instruction=SYSTEM_PROMPT
        )
        
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                # temperature=0.2,
                response_mime_type="application/json"
            )
        )
        
        gemini_response_raw_text = response.text
        
        return gemini_response_raw_text

    except json.JSONDecodeError as e:
        print(f"错误:无法解析 GPT 回复的 JSON(question_number '{original_question_number}'): {e}")
        if response_content:
            print(f"原始回复内容: {response_content}")
        else:
            print("未获取到原始回复内容。")
        return []
    except Exception as e:
        print(f"调用 API 过程中发生错误(question_number '{original_question_number}'): {e}")
        if response_content: 
            print(f"处理过程中的回复内容 (可能不完整或非JSON): {response_content}")
        return []


def save_checkpoint(all_transformed_questions, current_question_numberx):
    """保存当前处理进度和结果到checkpoint文件"""
    try:
        checkpoint_data = {
            'all_transformed_questions': all_transformed_questions,
            'current_question_numberx': current_question_numberx
        }
        np.save(CHECKPOINT_FILE, checkpoint_data)
        print(f"检查点已保存到 '{CHECKPOINT_FILE}'。当前进度: {current_question_numberx}/{len(original_questions)}")
    except Exception as e:
        print(f"保存检查点时发生错误: {e}")


def load_checkpoint():
    """加载之前的处理进度和结果"""
    try:
        if os.path.exists(CHECKPOINT_FILE):
            checkpoint = np.load(CHECKPOINT_FILE, allow_pickle=True).item()
            all_transformed = checkpoint.get('all_transformed_questions', [])
            current_question_numberx = checkpoint.get('current_question_numberx', 0)
            print(f"已加载检查点文件。已处理 {current_question_numberx} 个题目,已转换 {len(all_transformed)} 个结果。")
            return all_transformed, current_question_numberx
        else:
            print("未找到检查点文件,将从头开始处理。")
            return [], 0
    except Exception as e:
        print(f"加载检查点时发生错误: {e}")
        return [], 0


# 主程序
try:
    # 加载题目数据
    with open(INPUT_JSON_FILE, 'r', encoding='utf-8') as f:
        original_questions = json.load(f)
except FileNotFoundError:
    print(f"错误:未找到输入文件 '{INPUT_JSON_FILE}'。")
    exit(1)

# 加载上次处理结果
all_transformed_questions, start_question_numberx = load_checkpoint()
print(f"找到 {len(original_questions)} 个待处理题目。")

if start_question_numberx > 0 and start_question_numberx < len(original_questions):
    print(f"从索引 {start_question_numberx} 处恢复处理 (已完成 {start_question_numberx}/{len(original_questions)} 个题目)")
else:
    start_question_numberx = 0
    print("从头开始处理题目。")

# 处理题目
for question_numberx in range(start_question_numberx, len(original_questions)):
    problem_obj = original_questions[question_numberx]
    print(f"\n处理第 {question_numberx+1}/{len(original_questions)} 个题目 (question_number: {problem_obj.get('question_number')})...")

    flag = True
    count = 0

    while flag:
    
        try:
            print(f"尝试第 {count+1} 次")
            ans = call_gpt_for_transformation(problem_obj)
            
            ans_json = json.loads(ans)['result']

            
            
            all_transformed_questions.extend(ans_json)

            flag = False

            # 每处理5个题目保存一次检查点
            if (question_numberx + 1) % 5 == 0 or question_numberx == len(original_questions) - 1:
                save_checkpoint(all_transformed_questions, question_numberx)
                
        except Exception as e:
            count += 1
    
    break
        
# 处理完成后保存最终结果
if all_transformed_questions:
    try:
        with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_transformed_questions, f, ensure_ascii=False, indent=4)
        print(f"成功保存转换后题目至 '{OUTPUT_JSON_FILE}'。")
    except Exception as e:
        print(f"写入文件 '{OUTPUT_JSON_FILE}' 时发生错误:{e}")

print(f"转换完成！共处理 {len(original_questions)} 个题目,生成 {len(all_transformed_questions)} 个转换后的题目。")

c:\Users\11541\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


未找到检查点文件，将从头开始处理。
找到 98 个待处理题目。
从头开始处理题目。

处理第 1/98 个题目 (ID: 题 21)...
尝试第 1 次

--- 发送题目 ID '题 21' 给 GPT 进行转换 ---
